In [52]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
import os
from PIL import Image

from Config import Config
from Models import Tools

/home/wene/miniconda3/envs/mlenv/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [46]:
notes = pd.read_csv('CSVs/combined.csv')
preds = pd.read_csv('predictions_17december.csv')

preds['Image Name'] = preds['Image Name'].map(lambda s : os.path.splitext(s)[0])
notes['Image Name'] = notes['Image Name'].map(lambda s : os.path.splitext(s)[0])

notes = notes.groupby(['Image Name']).mean()

def get_image_shape(file_path):
    with Image.open(file_path) as img:
        return img.size

In [47]:
df = preds.set_index('Image Name').join(notes, how='inner', lsuffix='_predicted', rsuffix='_real')

In [48]:
shapes  = df.index.map(lambda s : get_image_shape(
    os.path.join(Config.images_folder_path, s + '.png')
    )
).to_frame( index=False,name=['X','Y']).values

df['X'] = shapes[:,0]
df['Y'] = shapes[:,1]

In [49]:
df = df.assign(
    x1_predicted=lambda d: d['x1_predicted'] / d['X'],
    y1_predicted=lambda d: d['y1_predicted'] / d['Y'],
    x2_predicted=lambda d: d['x2_predicted'] / d['X'],
    y2_predicted=lambda d: d['y2_predicted'] / d['Y'],
    x1_real=lambda d: d['x1_real'] / d['X'],
    y1_real=lambda d: d['y1_real'] / d['Y'],
    x2_real=lambda d: d['x2_real'] / d['X'],
    y2_real=lambda d: d['y2_real'] / d['Y'], 
).drop(['X','Y'], axis=1)

In [55]:
outputs = torch.tensor(df[['x1_predicted','y1_predicted','x2_predicted','y2_predicted']].values)
keypoints = torch.tensor(df[['x1_real','y1_real','x2_real','y2_real']].values)

clamp_threshold = 0.004

errors = (keypoints - outputs)**2
mse_per_vector = torch.mean(errors, dim=1, keepdim=True)
mse_per_vector_np = mse_per_vector.cpu().numpy()

# Step 1: Normalize by mean and variance
mean = mse_per_vector.mean()
std = mse_per_vector.std()
normalized_vector = (mse_per_vector - mean) / std

min_val = normalized_vector.min()
max_val = normalized_vector.max()

scaled_errors = (normalized_vector - min_val) / (max_val - min_val)

df['Error scale'] = scaled_errors

print(f"MSE loss {torch.nn.MSELoss()(outputs, keypoints)}")
print(f"Tools.py distance loss (normalized?) {Tools.get_distance_loss(outputs, keypoints)}")

MSE loss 0.045702337944739976
Tools.py distance loss (normalized?) 0.12729763552053083


In [37]:
print(f"The error mean is {df['Error scale'].mean()}")
print(f"The error variance is {df['Error scale'].var()}")

The error mean is 0.26820112739373847
The error variance is 0.010187071145561702
